In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import glob
import numpy as np
from deepdiff import DeepDiff

### Past decoding

In [5]:
peanut_decoding_files = glob.glob('/mnt/Secondary/data/peanut_decoding01/peanut_decoding01_*.dat')

In [8]:
result_list = []
for file in peanut_decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)

In [9]:
peanut_decoding_df = pd.DataFrame(result_list)

In [12]:
with open('/home/akumar/nse/neural_control/data/peanut_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(peanut_decoding_df))

In [13]:
sabes_decoding_files = glob.glob('/mnt/Secondary/data/sabes_decoding01/sabes_decoding01_*.dat')

In [18]:
result_list = []
for file in sabes_decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)

In [19]:
sabes_decoding_df = pd.DataFrame(result_list)

In [21]:
with open('/home/akumar/nse/neural_control/data/sabes_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(sabes_decoding_df))

### Conslidating 4/27

In [5]:
indyparametric_files = glob.glob('/mnt/Secondary/data/indy_dimreduc_parametric/indy_dimreduc_parametric_*.dat')

In [8]:
result_list = []
for file in indyparametric_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
dimreduc_df = pd.DataFrame(result_list)

In [9]:
with open('/home/akumar/nse/neural_control/data/indy_parametric_dimreduc_df.dat', 'wb') as f:
    f.write(pickle.dumps(dimreduc_df))

In [15]:
dimreduc_files = glob.glob('/mnt/Secondary/data/indy_trialized_dimreduc/indy_trialized_dimreduc_*.dat')

In [17]:
result_list = []
for file in dimreduc_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
dimreduc_df = pd.DataFrame(result_list)

In [18]:
with open('/home/akumar/nse/neural_control/data/indy_trialized_dimreduc_df.dat', 'wb') as f:
    f.write(pickle.dumps(dimreduc_df))

In [3]:
decoding_files = glob.glob('/mnt/Secondary/data/peanut_decoding_fcca/peanut_decoding_fcca_*.dat')

In [5]:
result_list = []
for file in decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [6]:
with open('/home/akumar/nse/neural_control/data/peanut_fcca_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(decoding_df))

In [3]:
decoding_files = glob.glob('/mnt/Secondary/data/indydimreducparam_decoding/indydimreducparam_decoding_*.dat')

In [4]:
result_list = []


result_list = []
for file in decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [5]:
import hickle

In [3]:
argfiles = glob.glob('/mnt/Secondary/data/indydimreducparam_decoding/arg*.dat')

In [4]:
loader_args = []
for argfile in argfiles:
    with open(argfile, 'rb') as f:
        args = pickle.load(f)
    
    # Track down the arg file associated with the dimreduc result
    dimreducpath = '/'.join(args['task_args']['dimreduc_file'].split('/')[:-1])
    dimreducno = args['task_args']['dimreduc_file'].split('.dat')[0].split('_')[-1]
    argfile = '%s/arg%s.dat' % (dimreducpath, dimreducno)    
    
    with open(argfile, 'rb') as f:
        dimreduc_args = pickle.load(f)
    
    loader_args.append(dimreduc_args['loader_args'])

In [5]:
unique_loader_args = []
for loader_arg in loader_args:
    if not unique_loader_args:
        unique_loader_args.append(loader_arg)
    else:
        diffs = []
        for ula in unique_loader_args:
            diffs.append(DeepDiff(ula, loader_arg))
        
        diffs = np.array([len(d) for d in diffs])
        if np.all(diffs > 0):
            unique_loader_args.append(loader_arg)
        

In [6]:
len(unique_loader_args)

16

In [7]:
import pdb

In [8]:
# Separately agregarate and save for each loader arg
for i, ula in enumerate(unique_loader_args):
    results = []
    for loader_arg, argfile in zip(loader_args, argfiles):
        if len(DeepDiff(loader_arg, ula)) == 0:
            # Open up the results file
            with open(argfile, 'rb') as f:
                args = pickle.load(f)

            results_file = args['results_file']
            with open(results_file, 'rb') as f:
                result = pickle.load(f)

            results.extend(result)

    df = pd.DataFrame(results)
    with open('/home/akumar/nse/neural_control/data/indy_parametric_decoding/df%d.dat' % i, 'wb') as f:
        f.write(pickle.dumps(df))

In [3]:
parametric_decoding_dfs = glob.glob('/home/akumar/nse/neural_control/data/indy_parametric_decoding/df*.dat')

In [10]:
with open(parametric_decoding_dfs[0], 'rb') as f:
    df = pickle.load(f)

for pdf in parametric_decoding_dfs[1:]:
    with open(pdf, 'rb') as f:
        df_ = pickle.load(f)
    df = pd.concat([df, df_])

In [12]:
df.iloc[0]

dim                                                               10
fold_idx                                                           0
train_idxs         [11800, 11801, 11802, 11803, 11804, 11805, 118...
test_idxs          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
dimreduc_method                                                  DCA
dimreduc_args                                  {'T': 3, 'n_init': 5}
coef               [[-0.02989129865821936, 0.009189640239225123, ...
score                                                      67.644525
decoder                                                           lr
decoder_args       {'trainlag': 0, 'testlag': 0, 'decoding_window...
decoder_obj                         LinearRegression(normalize=True)
r2                 [0.14737225261022546, 0.23704996645774157, 0.0...
Name: 0, dtype: object

In [3]:
decoding_files = glob.glob('/mnt/Secondary/data/indytrialized_decoding/indytrialized_decoding_*.dat')

In [4]:
result_list = []
for file in decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [5]:
with open('/home/akumar/nse/neural_control/data/indytrialized_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(decoding_df))